In [ ]:
import sys
import os
from pathlib import Path

# Caminho absoluto para a raiz do projeto (um nível acima de notebooks)
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)


In [ ]:
from config.utils import *


In [ ]:
for nome_moeda, parametros in list(INDICADORES.items()):
    if parametros['FONTE'] == 'IBGE':
        print(nome_moeda)

In [ ]:
path = os.path.join(PATHS['BRONZE_LAYER'],"ipca_20250701.json")

In [ ]:
with open(path, "r", encoding="utf-8") as f:
    dados = json.load(f)
    pd.DataFrame(dados[0]['resultados'][0]['series'][0]['serie'].items(),columns=['data','valor'])
    


In [ ]:
pd.DataFrame(dados[0]['resultados'][0]['series'][0]['serie'].items(),columns=['data','valor'])

In [ ]:
salvar_dados_indicadores

In [ ]:
for inidcador, parametros in list(INDICADORES.items()):
   if parametros['FONTE'] == 'BACEN':
      dados_indicador = ler_dados_indicadores(nome_indicador=inidcador, nome_fonte='BACEN')
      salvar_dados_indicadores(dados=dados_indicador,nome_indicador=inidcador)
    
   else:
     dados_indicador = ler_dados_indicadores(nome_indicador=inidcador, nome_fonte='IBGE')
     salvar_dados_indicadores(dados=dados_indicador,nome_indicador=inidcador)

In [ ]:
path_silver = os.path.join(PATHS['SILVER_LAYER'],"ipca.parquet")
t = pd.read_parquet(path_silver)

In [ ]:
t

In [ ]:
t['data'] = pd.to_datetime(t['data'].astype(str), format='%Y%m').dt.strftime('%d/%m/%Y')



In [ ]:
INDICADORES.keys()

In [ ]:
PATHS['BRONZE_LAYER']

In [ ]:
def ler_dados_cotacao(moeda: str, path: str = PATHS['BRONZE_LAYER'], data: str = DATA_FINAL):
    """
    Lê um arquivo JSON de cotação e retorna um DataFrame.

    Args:
        path (str): Caminho da pasta onde está o JSON.
        moeda (str): Nome da moeda (ex: 'USD_BRL').
        data (datetime, opcional): Data desejada. Default: DATA_FINAL.

    Returns:
        [pd.DataFrame]: DataFrame formatado ou None se houver erro.
    """

    try:
        moeda = moeda.lower()
        data_str = pd.to_datetime(data, format="%d/%m/%Y").strftime("%Y%m%d")
        nome_arquivo = f"{moeda}_{data_str}.json"
        path_arquivo = os.path.join(path, nome_arquivo)

        logger.info(f"Lendo arquivo: {path_arquivo}")

        df = pd.read_json(path_arquivo)

       
        df.rename(columns={'valor': moeda}, inplace=True)
        df = df.astype({'data': 'datetime64', moeda.lower(): 'float64'}, errors='ignore')

      
        return df

    except FileNotFoundError:
        logger.warning(f"Arquivo não encontrado: {nome_arquivo}")
    except Exception as e:
        logger.error(f"Erro ao ler {moeda} - {data_str}: {e}", exc_info=True)

    return None

In [ ]:
moedas = ['ARS_BRL', 'USD_BRL', 'EUR_BRL', 'GBP_BRL', 'JPY_BRL', 'CHF_BRL', 'CNY_BRL']

df_cotacoes = ler_dados_cotacao(moeda='ars_brl',path=PATHS['BRONZE_LAYER'])

In [ ]:
df_cotacoes

In [ ]:
def salvar_dados_cotacao(dados: pd.DataFrame, moeda: str):
    """
    Salva os dados brutos da API na camada silver.

    Args:
        dados (DataFrame): Dados retornados da API.
        nome_cotacao (str): Nome da moeda de cotação.

    Returns:
        None
    """
    try:
        nome_arquivo = f"{moeda.lower()}.parquet"
        caminho_arquivo = os.path.join(PATHS["SILVER_LAYER"],nome_arquivo)
        print(caminho_arquivo)
        dados.to_parquet(caminho_arquivo)

        
        logger.info(f"Indicacdor {moeda} salvo no arquivo: {nome_arquivo}")

    except Exception as e:
        logger.error(f"Erro ao salvar os dados na camada silver: {str(e)}")
        raise e

In [ ]:
salvar_dados_cotacao(df_cotacoes,'ARS_BRL')

In [ ]:
for nome_moeda, parametros in list(INDICADORES.items())[-7:]:
   dados_cotacao = ler_dados_cotacao(moeda=nome_moeda)
   salvar_dados_cotacao(dados=dados_cotacao,moeda=nome_moeda)


In [ ]:
teste = obter_dados_api(INDICADORES['IPCA'])

In [ ]:
#INDICADORES['IPCA']['URL'],
INDICADORES['DESEMPREGO']['URL']

In [ ]:
teste[0]['resultados'][0]['series'][0]['serie']

In [ ]:
salvar_dados_api(teste, 'ipca')

In [ ]:
for nome_indicador, config in INDICADORES.values():
    try:
        dados = obter_dados_api(indicador)
        salvar_dados_api(dados, indicador['NOME'])
    except Exception as e:
        logger.error(f"Erro ao processar o indicador {indicador['NOME']}: {str(e)}")

In [ ]:

Cotaçao cambio
response = requests.get("https://api.bcb.gov.br/dados/serie/bcdata.sgs.21619/dados?formato=json&dataInicial=29/06/2015")
response.json()

In [ ]:
url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.432/dados?formato=json&dataInicial=01/07/2015&dataFinal=01/07/2025"

response = requests.get(url)
response.raise_for_status()  
dados_cambio = response.json()

In [ ]:
dados_cambio